In [0]:
## Lens.org
UdeA: https://link.lens.org/DqI511uElPi

In [0]:
import pandas as pd

In [0]:
ln=pd.read_json('lensudea.json.gz',compression='gzip')

In [0]:
ln.shape

(23714, 25)

In [0]:
ln[:3]

,abstract,authors,chemicals,citation_ids,clinical_trials,date_published,fields_of_study,funding,issue,journal,...,publication_supplementary_type,publication_type,record_lens_id,referenced_by_count,referenced_by_patent_count,references,source_urls,title,volume,year_published
0,Stainless steel has been frequently used for t...,[{'last_affiliation_id': {'value': 'mag3596168...,"[{'substance_name': 'Coated Materials, Biocomp...","[{'value': '17122924', 'type': 'pmid'}, {'valu...",[],2006-11-22T00:00:00+00:00,"[{'name': 'Composite material', 'similarity': ...",[],11,"{'issn': [{'value': '09574530', 'type': 'print...",...,"[research support, non-u.s. gov't]",journal article,000-007-853-017-579,9,0,"[{'lens_id': 269340095748}, {'lens_id': 554471...",[{'url': 'https://link.springer.com/article/10...,Cytotoxicity study of plasma-sprayed hydroxyap...,17,2006
1,Speech disorders are among the most common sym...,[{'last_affiliation_id': {'value': 'mag3596168...,[],"[{'value': 'mag2739944706', 'type': 'magid'}, ...",[],2017-08-27T00:00:00+00:00,"[{'name': 'Czech', 'similarity': 0.44207096100...",[],None,"{'issn': [{'value': '03029743', 'type': 'print...",...,[],conference proceedings,000-012-385-217-540,1,0,"[{'lens_id': 11621625958}, {'lens_id': 9583187...",[{'url': 'http://dblp.uni-trier.de/db/conf/tsd...,TSD - Language Independent Assessment of Motor...,None,2017
2,None,[{'last_affiliation_id': {'value': 'mag3596168...,[],"[{'value': 'mag2307874066', 'type': 'magid'}]",[],1999-01-01T00:00:00+00:00,"[{'name': 'Philosophy', 'similarity': 0.253162...",[],89,NaN,...,[],None,000-012-800-591-40X,0,0,[],[{'url': 'https://biblat.unam.mx/es/revista/re...,Actividad de las enzimas ligninolíticas del Ph...,23,1999


# Affiliation analysis

In [0]:
i=200
print(i,end='\r')
print(i,end='\r')

In [0]:
# Extract dictionary columns from a list of dictionaries
key={}
key[0]='authors' # level 0. Must be an string
key[1]=['affiliations','first_name','last_name','initials']
def nesteddf(df,key):
    aff=pd.DataFrame()
    #split df
    dfy=df[~df[key[0]].isna()].reset_index(drop=True)
    dfn=df[df[key[0]].isna()].reset_index(drop=True)
    #dfy[key] is a list of dictionaries
    lenmax=dfy[key[0]].apply(len).max()
    for i in range(lenmax):
        if i%100==0:print(i,end='\r')
        i_aff=pd.DataFrame()
        #select dictionary i from list
        i_key=dfy[~dfy[key[0]].str[i].isna()].reset_index(drop=True)
        for k in key[1]:
            i_aff[k]=i_key[key[0]].str[i].apply( lambda x: x.get(k) )
        #i_aff now have key[1]
        aff=aff.append(i_aff).reset_index(drop=True)
    return aff

In [0]:
aff=nesteddf(ln,key)

## Extract affiliations

In [0]:
daff=aff[aff.affiliations.apply(len)>0].reset_index(drop=True)
daff['name_0']=daff.affiliations.str[0].apply( lambda x: x.get('name')  )

Drop repeated full names with same primary affiliation

In [0]:
daff=daff.drop_duplicates(['name_0','first_name','last_name','initials']).sort_values('last_name').reset_index(drop=True)

Number of "different" authors of the full set of papers: 

In [0]:
daff.shape

(46671, 6)

normalize authors

In [0]:
daff['full_name']=daff['first_name']+' '+daff['last_name']

Checking only primary affiliation

In [0]:
tmp=daff[daff.name_0=='Grupo México']
tmp.shape

(387, 6)

Check some author

In [0]:
daff[ ( daff.full_name.str.contains('Restrepo') ) & ( daff.full_name.str.contains('Diego') ) ].sort_values('full_name')

,affiliations,first_name,last_name,initials,name_0,full_name
34669,[{'grid': {'links': ['http://www.udea.edu.co/p...,A O Diego,Restrepo,AOD,University of Antioquia,A O Diego Restrepo
28948,"[{'grid': None, 'name': 'Universidad CES, Mede...",Diego A Restrepo,O,DAR,"Universidad CES, Medellín, Colombia",Diego A Restrepo O
33276,[{'grid': {'links': ['http://www.udea.edu.co/p...,Diego Alejandro Restrepo,Quintero,DAR,University of Antioquia,Diego Alejandro Restrepo Quintero
29198,"[{'grid': None, 'name': 'Universidad CES, Mede...",Diego Alveiro Restrepo,Ochoa,DAR,"Universidad CES, Medellín, Colombia",Diego Alveiro Restrepo Ochoa
29230,"[{'grid': None, 'name': 'Universidad de San Bu...",Diego Alveiro Restrepo,Ochoa,DAR,"Universidad de San Buenaventura, Medellín",Diego Alveiro Restrepo Ochoa
34802,[{'grid': {'links': ['http://www.udea.edu.co/p...,Diego Alveiro,Restrepo-Ochoa,DA,University of Antioquia,Diego Alveiro Restrepo-Ochoa
34390,[{'grid': {'links': ['http://www.udea.edu.co/p...,Diego,Restrepo,D,University of Antioquia,Diego Restrepo
34645,"[{'grid': {'links': ['http://www.csic.es/'], '...",Diego,Restrepo,D,Spanish National Research Council,Diego Restrepo
34745,"[{'grid': {'links': ['http://www.ictp.it/'], '...",Diego,Restrepo,D,International Centre for Theoretical Physics,Diego Restrepo
34671,[{'grid': {'links': ['http://www.udea.edu.co/p...,Juan Diego Cardona,Restrepo,JDC,University of Antioquia,Juan Diego Cardona Restrepo


In [0]:
daff[ ( daff.full_name.str.contains('D[\.\s]Restrepo') ) & ( daff.full_name.str.contains('^D[\.\s]') ) ].sort_values('full_name')

,affiliations,first_name,last_name,initials,name_0,full_name
34310,[{'grid': {'links': ['http://www.udea.edu.co/p...,D,Restrepo,D,University of Antioquia,D Restrepo


Search all declarated affillations

In [0]:
key={}
key[0]='affiliations' # level 0. Must be an string
key[1]=['grid','name','name_raw']
aff=nesteddf(daff,key)

In [0]:
aff[:3]

,grid,name,name_raw
0,{'links': ['http://www.udea.edu.co/portal/page...,University of Antioquia,"Universidad de Antioquia, Medellín, Colombia"
1,{'links': ['http://www.udea.edu.co/portal/page...,University of Antioquia,Universidad de Antioquia
2,{'links': ['http://www.udea.edu.co/portal/page...,University of Antioquia,"Universidad de Antioquia, Medellín, Colombia"


In [0]:
aff [ ( aff.name=='University of Antioquia' ) & ( 
    (aff.name_raw.str.contains('F[íi]sica')) |  (aff.name_raw.str.contains('Physics'))
) ].shape

(265, 3)

In [0]:
aff [ ( aff.name=='University of Antioquia' ) & ( aff.name_raw.str.contains('Physics') ) ].s

,grid,name,name_raw
1188,{'links': ['http://www.udea.edu.co/portal/page...,University of Antioquia,"Solar, Earth and Planetary Physics Group (SEAP..."
1202,{'links': ['http://www.udea.edu.co/portal/page...,University of Antioquia,"Department of Physics, University of Antioquia..."
1367,{'links': ['http://www.udea.edu.co/portal/page...,University of Antioquia,"Institute of Physics, University of Antioquia,..."
2148,{'links': ['http://www.udea.edu.co/portal/page...,University of Antioquia,"Institute of Physics, Faculty of Exact and Nat..."
2592,{'links': ['http://www.udea.edu.co/portal/page...,University of Antioquia,"Institute of Physics, University of Antioquia,..."
3566,{'links': ['http://www.udea.edu.co/portal/page...,University of Antioquia,"Solid State Group, Department of Physics, Univ..."
6505,{'links': ['http://www.udea.edu.co/portal/page...,University of Antioquia,"Department of Physics, Universidad del Valle, ..."
8008,{'links': ['http://www.udea.edu.co/portal/page...,University of Antioquia,"Institute of Physics, University of Antioquia,..."
8068,{'links': ['http://www.udea.edu.co/portal/page...,University of Antioquia,"Institute of Physics, University of Antioquia,..."
11688,{'links': ['http://www.udea.edu.co/portal/page...,University of Antioquia,"Institut of Physics, Universidad de Antioquia,..."


In [0]:
rdf=pd.DataFrame()

In [0]:
rdf['institution']=aff.name.value_counts().keys()

In [0]:
rdf['colaborators']=aff.name.value_counts().values

Recover full information of institution

In [0]:
rdf=rdf.merge(aff[['grid','name']].drop_duplicates('name').reset_index(drop=True),
          left_on='institution',right_on='name',how='left').drop('name',axis='columns')

In [0]:
rdf[:1]

,institution,colaborators,grid
0,University of Antioquia,17564,{'links': ['http://www.udea.edu.co/portal/page...


In [0]:
key={}
key[0]='grid'
key[1]=['addresses','wikipedia_url','id','links','types']
def extract_nested_columns(df,key,col_type='dict'):
    import sys
    if col_type=='dict':
        sdf=df[key[0]]
    elif col_type=='list':
        sdf=df[key[0]].str[0]
    else:
        sys.exit('bad column type')
    dfy=df[~sdf.isna()].reset_index(drop=True)
    dfn=df[ sdf.isna()].reset_index(drop=True)
    for k in key[1]:
        if col_type=='dict':
            sdfy=dfy[key[0]]
        elif col_type=='list':
            sdfy=dfy[key[0]].str[0]            
        
        dfy[k]=sdfy.apply( lambda x: x.get(k) )
        dfn[k]=None
    dft=dfy.append(dfn).reset_index(drop=True)
    return dft

In [0]:
df=extract_nested_columns(rdf,key)

In [0]:
df[:2]

,institution,colaborators,grid,addresses,wikipedia_url,id,links,types
0,University of Antioquia,17564,{'links': ['http://www.udea.edu.co/portal/page...,"[{'state_code': '', 'lat': 6.267417, 'country_...",http://en.wikipedia.org/wiki/University_of_Ant...,grid.412881.6,[http://www.udea.edu.co/portal/page/portal/Eng...,[Education]
1,National University of Colombia,892,"{'links': ['http://unal.edu.co/'], 'wikipedia_...","[{'state_code': '', 'lat': 4.635556, 'country_...",http://en.wikipedia.org/wiki/National_Universi...,grid.10689.36,[http://unal.edu.co/],[Education]


In [0]:
key={}
key[0]='addresses'
key[1]=['city','country_code','lat','lon']
dff=extract_nested_columns(df,key,col_type='list')

In [0]:
dff=dff.drop(['grid','addresses'],axis='columns')

In [0]:
dff

,institution,colaborators,wikipedia_url,id,links,types,city,country_code,lat,lon
0,University of Antioquia,17564,http://en.wikipedia.org/wiki/University_of_Ant...,grid.412881.6,[http://www.udea.edu.co/portal/page/portal/Eng...,[Education],Medellín,CO,6.26742,-75.5684
1,National University of Colombia,892,http://en.wikipedia.org/wiki/National_Universi...,grid.10689.36,[http://unal.edu.co/],[Education],Bogotá,CO,4.63556,-74.0828
2,Pontifical Bolivarian University,448,http://en.wikipedia.org/wiki/Pontifical_Boliva...,grid.412249.8,[http://www.upb.edu.co/portal/page?_pageid=954...,[Education],Medellín,CO,6.24245,-75.5898
3,CERN,251,https://en.wikipedia.org/wiki/CERN,grid.9132.9,[http://home.web.cern.ch/],[Facility],Geneva,CH,46.2342,6.05278
4,Harvard University,222,http://en.wikipedia.org/wiki/Harvard_University,grid.38142.3c,[http://www.harvard.edu/],[Education],Cambridge,US,42.3771,-71.1167
5,Spanish National Research Council,212,http://en.wikipedia.org/wiki/Spanish_National_...,grid.4711.3,[http://www.csic.es/],[Government],Madrid,ES,40.5959,-3.68818
6,University of Valle,200,http://en.wikipedia.org/wiki/University_of_Valle,grid.8271.c,[http://www.univalle.edu.co/],[Education],Santiago de Cali,CO,3.37511,-76.5335
7,University of São Paulo,196,http://en.wikipedia.org/wiki/University_of_S%C...,grid.11899.38,[http://www5.usp.br/en/],[Education],São Paulo,BR,-23.5631,-46.7301
8,Pontifical Xavierian University,192,http://en.wikipedia.org/wiki/Pontifical_Xavier...,grid.41312.35,[http://www.javeriana.edu.co/home],[Education],Bogotá,CO,4.62894,-74.0649
9,National Institutes of Health,159,http://en.wikipedia.org/wiki/National_Institut...,grid.94365.3d,[http://www.nih.gov/],[Government],Bethesda,US,39.0004,-77.1024


In [0]:
dff[1:].to_excel('colab.xlsx',index=False)

In [0]:
pd.read_excel('colab.xlsx')

,institution,colaborators,wikipedia_url,id,links,types,city,country_code,lat,lon
0,National University of Colombia,892,http://en.wikipedia.org/wiki/National_Universi...,grid.10689.36,['http://unal.edu.co/'],['Education'],Bogotá,CO,4.635556,-74.082778
1,Pontifical Bolivarian University,448,http://en.wikipedia.org/wiki/Pontifical_Boliva...,grid.412249.8,['http://www.upb.edu.co/portal/page?_pageid=95...,['Education'],Medellín,CO,6.242450,-75.589833
2,CERN,251,https://en.wikipedia.org/wiki/CERN,grid.9132.9,['http://home.web.cern.ch/'],['Facility'],Geneva,CH,46.234167,6.052778
3,Harvard University,222,http://en.wikipedia.org/wiki/Harvard_University,grid.38142.3c,['http://www.harvard.edu/'],['Education'],Cambridge,US,42.377053,-71.116657
4,Spanish National Research Council,212,http://en.wikipedia.org/wiki/Spanish_National_...,grid.4711.3,['http://www.csic.es/'],['Government'],Madrid,ES,40.595884,-3.688175
5,University of Valle,200,http://en.wikipedia.org/wiki/University_of_Valle,grid.8271.c,['http://www.univalle.edu.co/'],['Education'],Santiago de Cali,CO,3.375109,-76.533536
6,University of São Paulo,196,http://en.wikipedia.org/wiki/University_of_S%C...,grid.11899.38,['http://www5.usp.br/en/'],['Education'],São Paulo,BR,-23.563051,-46.730102
7,Pontifical Xavierian University,192,http://en.wikipedia.org/wiki/Pontifical_Xavier...,grid.41312.35,['http://www.javeriana.edu.co/home'],['Education'],Bogotá,CO,4.628944,-74.064850
8,National Institutes of Health,159,http://en.wikipedia.org/wiki/National_Institut...,grid.94365.3d,['http://www.nih.gov/'],['Government'],Bethesda,US,39.000443,-77.102394
9,Imperial College London,157,http://en.wikipedia.org/wiki/Imperial_College_...,grid.7445.2,['http://www3.imperial.ac.uk/'],['Education'],London,GB,51.498600,-0.175478


In [0]:
dff.country_code.value_counts()

US    444
GB    119
DE     69
ES     65
FR     60
CA     57
AU     54
IT     52
BR     49
CO     46
CN     46
JP     46
IN     37
TR     32
MX     30
CH     24
NL     24
AR     23
BE     17
SE     17
IL     17
RU     16
AT     16
CL     16
KR     16
ZA     13
NO     13
IR     13
PL     12
FI     12
     ... 
IQ      1
MN      1
LT      1
DZ      1
LU      1
BO      1
LA      1
SY      1
SZ      1
CY      1
CM      1
MM      1
BH      1
JE      1
IS      1
PS      1
SV      1
JO      1
PG      1
ME      1
BS      1
SR      1
ZW      1
TN      1
KH      1
BY      1
UA      1
BJ      1
OM      1
PY      1
Name: country_code, Length: 108, dtype: int64

## Checks

In [0]:
dff[dff.country_code=='CO'].reset_index(drop=True)

,institution,colaborators,wikipedia_url,id,links,types,city,country_code,lat,lon
0,University of Antioquia,17564,http://en.wikipedia.org/wiki/University_of_Ant...,grid.412881.6,[http://www.udea.edu.co/portal/page/portal/Eng...,[Education],Medellín,CO,6.26742,-75.5684
1,National University of Colombia,892,http://en.wikipedia.org/wiki/National_Universi...,grid.10689.36,[http://unal.edu.co/],[Education],Bogotá,CO,4.63556,-74.0828
2,Pontifical Bolivarian University,448,http://en.wikipedia.org/wiki/Pontifical_Boliva...,grid.412249.8,[http://www.upb.edu.co/portal/page?_pageid=954...,[Education],Medellín,CO,6.24245,-75.5898
3,University of Valle,200,http://en.wikipedia.org/wiki/University_of_Valle,grid.8271.c,[http://www.univalle.edu.co/],[Education],Santiago de Cali,CO,3.37511,-76.5335
4,Pontifical Xavierian University,192,http://en.wikipedia.org/wiki/Pontifical_Xavier...,grid.41312.35,[http://www.javeriana.edu.co/home],[Education],Bogotá,CO,4.62894,-74.0649
5,EAFIT University,127,https://en.wikipedia.org/wiki/EAFIT_University,grid.448637.a,[http://www.eafit.edu.co/Paginas/index.aspx],[Education],Medellín,CO,6.20081,-75.5785
6,University of Los Andes,123,http://en.wikipedia.org/wiki/University_of_Los...,grid.7247.6,[http://www.uniandes.edu.co/],[Education],Bogotá,CO,4.60161,-74.0652
7,Universidad del Tolima,66,http://en.wikipedia.org/wiki/University_of_Tolima,grid.412192.d,[http://www.ut.edu.co/],[Education],Ibagué,CO,4.42751,-75.2131
8,University of Caldas,55,None,grid.7779.e,None,[Education],Manizales,CO,NaN,NaN
9,University of Cartagena,52,http://en.wikipedia.org/wiki/University_of_Car...,grid.412885.2,[http://www.unicartagena.edu.co/],[Education],Cartagena,CO,10.4025,-75.5047


In [0]:
autores_udea=daff[daff.name_0=='University of Antioquia'].reset_index(drop=True)

In [0]:
autores_udea_norm=autores_udea[~autores_udea.first_name.isna()].reset_index(drop=True)
autores_udea_norm=autores_udea_norm[~autores_udea_norm.last_name.isna()].reset_index(drop=True)

In [0]:
autores_udea_norm[autores_udea_norm.first_name.str.contains('Restrepo')].reset_index(drop=True)[29:]

,affiliations,first_name,last_name,initials,name_0
29,[{'grid': {'wikipedia_url': 'http://en.wikiped...,Juan Felipe Restrepo,David,JFR,University of Antioquia
30,[{'grid': {'wikipedia_url': 'http://en.wikiped...,Sara L Restrepo,E,SLR,University of Antioquia
31,[{'grid': {'wikipedia_url': 'http://en.wikiped...,Mauricio Restrepo,E,MR,University of Antioquia
32,[{'grid': {'wikipedia_url': 'http://en.wikiped...,María Isabel Restrepo,E.,MIR,University of Antioquia
33,[{'grid': {'wikipedia_url': 'http://en.wikiped...,Tulio Restrepo,Echeverri,TR,University of Antioquia
34,[{'grid': {'wikipedia_url': 'http://en.wikiped...,M. Restrepo,Escobar,MR,University of Antioquia
35,[{'grid': {'wikipedia_url': 'http://en.wikiped...,Francisco Restrepo,Escobar,FR,University of Antioquia
36,[{'grid': {'wikipedia_url': 'http://en.wikiped...,Mauricio Restrepo,Escobar,MR,University of Antioquia
37,[{'grid': {'wikipedia_url': 'http://en.wikiped...,Restrepo,Espinosa,R,University of Antioquia
38,[{'grid': {'wikipedia_url': 'http://en.wikiped...,Diana Carolina Restrepo,Espinosa,DCR,University of Antioquia


In [0]:
daff[daff.name_0!='University of Antioquia'][:10]

,affiliations,first_name,last_name,initials,name_0
13,"[{'grid': None, 'ids': [], 'name': 'señor'}]",José M. Begiristain,A,JMB,señor
21,"[{'grid': None, 'ids': [], 'name': 'Universida...",Doris Cardona,A,DC,Universidad CES
25,"[{'grid': None, 'ids': [], 'name': 'señor'}]",Andrés José Álvarez,A,AJÁ,señor
29,[{'grid': {'wikipedia_url': 'http://en.wikiped...,Vortmeyer,A,V,National Institutes of Health
32,"[{'grid': None, 'ids': [], 'name': 'Universida...",Carmen Gloria Artigas,A,CGA,"Universidad San Sebastián, Santiago de Chile"
33,[{'grid': {'wikipedia_url': 'http://en.wikiped...,Bohórquez Peñaranda,A,BP,Pontifical Xavierian University
34,"[{'grid': None, 'ids': [], 'name': 'Universida...",Ernesto García,A,EG,"Universidad Industrial de Santander, Colombia"
35,"[{'grid': None, 'ids': [{'type': 'magid', 'val...",Gabriel J. Arango,A,GJA,Grupo México
36,[{'grid': {'wikipedia_url': 'http://en.wikiped...,Lanoue,A,L,François Rabelais University
38,"[{'grid': None, 'ids': [], 'name': 'señor'}]",Jorge Alberto Moreno,A,JAM,señor


In [0]:
autores_udea_norm[( autores_udea_norm.last_name.str.contains('Restrepo') ) & 
                 ( autores_udea_norm.first_name.str.contains('D') )].reset_index(drop=True)

,affiliations,first_name,last_name,initials,name_0
0,[{'grid': {'wikipedia_url': 'http://en.wikiped...,Daniel,Alonso-Restrepo,D,University of Antioquia
1,[{'grid': {'wikipedia_url': 'http://en.wikiped...,Daniel,Correa-Restrepo,D,University of Antioquia
2,[{'grid': {'wikipedia_url': 'http://en.wikiped...,Diana,Granada-Restrepo,D,University of Antioquia
3,[{'grid': {'wikipedia_url': 'http://en.wikiped...,Diana María,Granda-Restrepo,DM,University of Antioquia
4,[{'grid': {'wikipedia_url': 'http://en.wikiped...,D,Restrepo,D,University of Antioquia
5,[{'grid': {'wikipedia_url': 'http://en.wikiped...,V Diego,Restrepo,VD,University of Antioquia
6,[{'grid': {'wikipedia_url': 'http://en.wikiped...,Oriana Delgado,Restrepo,OD,University of Antioquia
7,[{'grid': {'wikipedia_url': 'http://en.wikiped...,Diana,Restrepo,D,University of Antioquia
8,[{'grid': {'wikipedia_url': 'http://en.wikiped...,Kevin Daniel,Restrepo,KD,University of Antioquia
9,[{'grid': {'wikipedia_url': 'http://en.wikiped...,P Gabriel Darío,Restrepo,PGD,University of Antioquia


In [0]:
autores_udea_norm[( autores_udea_norm.first_name.str.contains('Restrepo') ) & 
                 ( autores_udea_norm.first_name.str.contains('D') )].reset_index(drop=True)

,affiliations,first_name,last_name,initials,name_0
0,[{'grid': {'wikipedia_url': 'http://en.wikiped...,Karen Daniela Restrepo,Arboleda,KDR,University of Antioquia
1,[{'grid': {'wikipedia_url': 'http://en.wikiped...,Diana María Restrepo,Arias,DMR,University of Antioquia
2,[{'grid': {'wikipedia_url': 'http://en.wikiped...,Rubén Darío Restrepo,Avendaño,RDR,University of Antioquia
3,[{'grid': {'wikipedia_url': 'http://en.wikiped...,Diana Carolina Restrepo,Espinosa,DCR,University of Antioquia
4,[{'grid': {'wikipedia_url': 'http://en.wikiped...,Daniela Restrepo,Guzmán.,DR,University of Antioquia
5,[{'grid': {'wikipedia_url': 'http://en.wikiped...,Duvan Restrepo,Martínez,DR,University of Antioquia
6,[{'grid': {'wikipedia_url': 'http://en.wikiped...,Diana Constanza Restrepo,O.,DCR,University of Antioquia
7,[{'grid': {'wikipedia_url': 'http://en.wikiped...,Diana Constanza Restrepo,Ochoa,DCR,University of Antioquia
8,[{'grid': {'wikipedia_url': 'http://en.wikiped...,D.P. Restrepo,Ospina,DPR,University of Antioquia
9,[{'grid': {'wikipedia_url': 'http://en.wikiped...,G.D Restrepo,Posada,GDR,University of Antioquia


In [0]:
aff.grid[0]

{'addresses': [{'city': 'Medellín',
   'country_code': 'CO',
   'lat': 6.267417,
   'lon': -75.568389,
   'state_code': ''}],
 'email_address': None,
 'id': 'grid.412881.6',
 'links': ['http://www.udea.edu.co/portal/page/portal/EnglishPortal/EnglishPortal'],
 'name': 'University of Antioquia',
 'status': 'active',
 'types': ['Education'],
 'wikipedia_url': 'http://en.wikipedia.org/wiki/University_of_Antioquia'}

In [0]:
aff[:1].grid.apply( lambda x: x.get('addresses') )

0    [{'lon': -75.568389, 'country_code': 'CO', 'st...
Name: grid, dtype: object

In [0]:
# Extract dictionary columns from a list of dictionaries
key={}
key[0]='authors' # level 0. Must be an string
key[1]=['affiliations','first_name','last_name','initials']
df=ln
aff=pd.DataFrame()
if True:
    #split df
    dfy=df[~df[key[0]].isna()].reset_index(drop=True)
    dfn=df[df[key[0]].isna()].reset_index(drop=True)
    #dfy[key] is a list of dictionaries
    lenmax=dfy[key[0]].apply(len).max()
    for i in range(lenmax):
        print(i)
        i_aff=pd.DataFrame()
        #select dictionary i from list
        i_key=dfy[~dfy[key[0]].str[i].isna()].reset_index(drop=True)
        for k in key[1]:
            i_aff[k]=i_key[key[0]].str[i].apply( lambda x: x.get(k) )
        #i_aff now have key[1]
        aff=aff.append(i_aff).reset_index(drop=True)

(46671, 6)

In [0]:
i_aff['affiliations'][0]

[{'grid': {'addresses': [{'city': 'Louvain-la-Neuve',
     'country_code': 'BE',
     'lat': 50.669611,
     'lon': 4.612264,
     'state_code': ''}],
   'email_address': None,
   'id': 'grid.7942.8',
   'links': ['http://www.uclouvain.be/en-index.html'],
   'name': 'Université Catholique de Louvain',
   'status': 'active',
   'types': ['Education'],
   'wikipedia_url': 'http://en.wikipedia.org/wiki/Universit%C3%A9_catholique_de_Louvain'},
  'ids': [{'type': 'magid', 'value': 'mag95674353'},
   {'type': 'grid', 'value': 'grid.7942.8'},
   {'type': 'isni', 'value': '000000012294713X'},
   {'type': 'fundref', 'value': '100007370'},
   {'type': 'fundref', 'value': '501100005041'},
   {'type': 'fundref', 'value': '501100005043'},
   {'type': 'fundref', 'value': '501100008135'},
   {'type': 'fundref', 'value': '100007355'},
   {'type': 'orgref', 'value': '398478'},
   {'type': 'wikidata', 'value': 'q378134'}],
  'name': 'Université catholique de Louvain'}]

In [0]:
i_aff[5:6]

,affiliations,first_name,last_name,initials
5,"[{'grid': None, 'ids': [], 'name': 'Museum of ...",Maria Martha,Dios,MM


In [0]:
lenmax=dfy[key].apply(len).max()
for i in [100]:#range(1):
    s=dfy[key].str[i]#.apply( lambda x: x.get(getkey) )

In [0]:
len( dfy[~dfy[key].str[i].isna()][key][553] )

2213

In [0]:
ln['aff']=ln.authors.str[0].apply(lambda x: x.get('affiliations')).str[0]

In [0]:
lny=ln[~ln.aff.isna()].reset_index(drop=True)
lnn=ln[ln.aff.isna()].reset_index(drop=True)

In [0]:
lnn['aff_name']=None

In [0]:
lny['aff_name']=lny.aff.apply(lambda x: x.get('name'))

In [0]:
ln=lny.append(lny).reset_index(drop=True)

In [0]:
ln.aff.apply(lambda x: x.get('grid'))[:6].apply(lambda x: x.get('addresses'))

AttributeError: 'NoneType' object has no attribute 'get'

In [0]:
x={'ids': [], 'grid': None}

In [0]:
x.get('grid') is None

True

In [0]:
ln.aff[:6]

0    {'ids': [{'value': 'mag35961687', 'type': 'mag...
1    {'ids': [{'value': 'mag35961687', 'type': 'mag...
2    {'ids': [{'value': 'mag35961687', 'type': 'mag...
3    {'ids': [{'value': 'mag16362100', 'type': 'mag...
4    {'ids': [{'value': 'mag35961687', 'type': 'mag...
5    {'ids': [], 'grid': None, 'name': 'Corporación...
Name: aff, dtype: object

In [0]:
lnn=ln[ln.aff.apply(lambda x: x.get('grid') is None)].reset_index(drop=True)
lny=ln[~ln.aff.apply(lambda x: x.get('grid') is None)].reset_index(drop=True)

In [0]:
lny['country_code']=lny.aff.apply(
    lambda x: x.get('grid') ).apply(
    lambda x: x.get('addresses') ).str[0].apply(
    lambda x: x.get('country_code') )

In [0]:
lny.aff.apply(
    lambda x: x.get('grid') ).apply(
    lambda x: x.get('addresses') )

0        [{'country_code': 'CO', 'lat': 6.267417, 'stat...
1        [{'country_code': 'CO', 'lat': 6.267417, 'stat...
2        [{'country_code': 'CO', 'lat': 6.267417, 'stat...
3        [{'country_code': 'IL', 'lat': 32.8684, 'state...
4        [{'country_code': 'CO', 'lat': 6.267417, 'stat...
5        [{'country_code': 'CO', 'lat': 6.267417, 'stat...
6        [{'country_code': 'CO', 'lat': 6.267417, 'stat...
7        [{'country_code': 'CO', 'lat': 6.267417, 'stat...
8        [{'country_code': 'CO', 'lat': 6.267417, 'stat...
9        [{'country_code': 'CO', 'lat': 6.267417, 'stat...
10       [{'country_code': 'CO', 'lat': 6.267417, 'stat...
11       [{'country_code': 'CO', 'lat': 6.267417, 'stat...
12       [{'country_code': 'CO', 'lat': 6.267417, 'stat...
13       [{'country_code': 'CO', 'lat': 6.267417, 'stat...
14       [{'country_code': 'CO', 'lat': 6.267417, 'stat...
15       [{'country_code': 'CO', 'lat': 6.267417, 'stat...
16       [{'country_code': 'CO', 'lat': 6.267417, 'stat.

In [0]:
ln.citation_ids[12]

[{'type': 'doi', 'value': '10.1007/978-0-85729-024-3_58'},
 {'type': 'magid', 'value': 'mag39991164'}]

In [0]:
ln.title[12]

'ISPE CE - Context Model for Testing Analysis Phase of Information Systems'

In [0]:
ln.references[12]

[{'lens_id': 1461871797126},
 {'lens_id': 3475226897286},
 {'lens_id': 5219935375808},
 {'lens_id': 6138891563925},
 {'lens_id': 6457769243819},
 {'lens_id': 7847830510193},
 {'lens_id': 9309225081657},
 {'lens_id': 9496614371740},
 {'lens_id': 11466114976775},
 {'lens_id': 19469456393751}]

In [0]:
ln[ln.record_lens_id.str.contains('269-340-095-748')]

,abstract,authors,chemicals,citation_ids,clinical_trials,date_published,fields_of_study,funding,issue,journal,...,publication_supplementary_type,publication_type,record_lens_id,referenced_by_count,referenced_by_patent_count,references,source_urls,title,volume,year_published


In [0]:
pd.DataFrame( pd.DataFrame( ln.authors[0] ).affiliations[0] ).grid[0]

{'addresses': [{'city': 'Medellín',
   'country_code': 'CO',
   'lat': 6.267417,
   'lon': -75.568389,
   'state_code': ''}],
 'email_address': None,
 'id': 'grid.412881.6',
 'links': ['http://www.udea.edu.co/portal/page/portal/EnglishPortal/EnglishPortal'],
 'name': 'University of Antioquia',
 'status': 'active',
 'types': ['Education'],
 'wikipedia_url': 'http://en.wikipedia.org/wiki/University_of_Antioquia'}